### MySQL

In [2]:
# Importing libraries and reading wiki_edit.txt file  
import pandas as p
import mysql.connector
from mysql.connector import errorcode

with open("wiki_edit.txt", "r", encoding= "UTF-8") as f:
    data = []
    for line in f:
        data.append(line.strip().split(' '))

data = p.DataFrame(data , columns = ['RevisionId', 'ArticleName', 'Timestamp', 'Editor'])
data[:5]

,RevisionId,ArticleName,Timestamp,Editor
0,9019643,Anarchism,2005-01-01,ip:24.136.172.37
1,9062018,Anarchism,2005-01-03,ip:137.53.22.23
2,9085524,Anarchism,2005-01-03,ip:24.42.90.254
3,9140836,Anarchism,2005-01-06,ip:69.61.184.201
4,9160986,A,2005-01-06,Fvw


In [3]:
# Credentials
uname = "jainadi5"
pwd = "A81237526" # Replaced by X after executing code 
hname = "mysql-user-cse.egr.msu.edu"

In [4]:

try:
    cnx = mysql.connector.connect(user=uname, password=pwd, host=hname, database=uname)
    cursor = cnx.cursor()
    query = "DROP TABLE IF EXISTS Wiki_Edit"
    cursor.execute(query)

    query = """CREATE TABLE IF NOT EXISTS Wiki_Edit (RevId int PRIMARY KEY,
                                ArticleName varchar(200),EditDate date,User varchar(50))
            """
    cursor.execute(query)
    cursor.close()
    cnx.commit()
    cnx.close()
    
except:
    print(mysql.connector.Error)

# Varchar(100) changed to varchar (200) for ArticleName due to presence of large size of article name in the database 

In [5]:
# Checking unicodes for alphanumeric range and replacing others with null
def byUnicode(char):
    if ord(char) < 48 or ord(char) > 127: return ''
    else: return char

In [6]:
# Inserting the data in the database row by row
try:
    cnx = mysql.connector.connect(user=uname, password=pwd, host=hname, database=uname)
    cursor = cnx.cursor()
    
    for index, row in data.iterrows():
        rid = row['RevisionId']
        Article = ''.join(list(filter(byUnicode, row['ArticleName'])))
        EditDate = row['Timestamp']
        User = ''.join(list(filter(byUnicode, row['Editor'])))
        query = "INSERT INTO Wiki_Edit VALUES ('" + rid + "','" + Article + "','" + EditDate + "','"+ User + "')"
        cursor.execute(query)
    cnx.commit()
    cursor.close()
    cnx.close()

except:
    print(mysql.connector.Error)   

In [68]:
# Selelcting the most revied article from the dataset
try:
    cnx = mysql.connector.connect(user=uname, password=pwd, host=hname, database=uname)
    cursor = cnx.cursor()
    query = "SELECT RevId, ArticleName, count(*) FROM Wiki_Edit GROUP BY ArticleName ORDER BY count(*) DESC LIMIT 1"
    cursor.execute(query)
    
    for (rid, article, count) in cursor:
        print (rid,article, count)   

    cursor.close()
    cnx.close()
    
except:
    print(mysql.connector.Error) 

8999630 2004_Indian_Ocean_earthquake 186


### MongoDb

In [1]:
# Importing libraries and creating client
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

In [2]:
# Creating a database/ collection by name wiki
db = client['wiki']

In [3]:
# Checking document counts
collection = db.wiki
collection.estimated_document_count()

0

In [4]:
# Adding the file into the document
with open("wiki_edit.txt", "r", encoding= "UTF-8") as f:
    for line in f:
        columns = line.strip().split(' ')
        record = { 'RevisionID': columns[0],'ArticleName': columns[1],'EditDate': columns[2], 'UserName': columns[3]}
        collection.insert_one(record)

In [5]:
# Finding details from articlename using .find
for line in collection.find({"ArticleName": "Anarchism"}):
    print(line["EditDate"], line["UserName"])

2005-01-01 ip:24.136.172.37
2005-01-03 ip:137.53.22.23
2005-01-03 ip:24.42.90.254
2005-01-06 ip:69.61.184.201


In [7]:
collection.estimated_document_count()

28209

In [6]:
client.close()